In [9]:
import geopandas as gpd
import pandas as pd
import classes.entropycalculator as ec
from scipy.stats import entropy
import numpy as np
import gc
import shapely
from tqdm import tqdm
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

tqdm.pandas()

part = 0

bu = gpd.read_parquet(f'data/buurten_parts/buurten_c_{part}.parquet')

categorisation = pd.read_excel('data/categorisation.xlsx')
L0_cats = categorisation['L0 category'].unique()
L1_cats = categorisation['L1 category'].unique()

In [10]:
def _get_shannon_entropy(labels, base=2):
    # get the total count of the labels
    total_count = len(labels)
    # get the unique labels and their counts
    _, label_counts = np.unique(labels, return_counts=True)

    probs = label_counts / total_count
    # get the entropy
    return entropy(probs, base=base)

In [11]:
bu.head(3)

,buurtcode,buurtnaam,wijkcode,gemeentecode,gemeentenaam,IND_WBI,H2O,POSTCODE,DEK_PERC,OAD,...,total_amenities_1,L0_leibovici_1,L1_leibovici_1,L0_altieri_1,L1_altieri_1,total_amenities_2,L0_leibovici_2,L1_leibovici_2,L0_altieri_2,L1_altieri_2
0,BU00140000,Binnenstad-Noord,WK001400,GM0014,Groningen,1.0,NEE,9712,1.0,6865.0,...,367.0,3.275661,6.925058,4.115122,7.973281,257.0,3.588746,6.602231,4.323108,7.367234
1,BU00140001,Binnenstad-Zuid,WK001400,GM0014,Groningen,1.0,NEE,9711,1.0,6586.0,...,601.0,3.346725,7.077308,4.438038,8.424578,463.0,3.559724,6.716213,4.644332,7.964978
2,BU00140002,Binnenstad-Oost,WK001400,GM0014,Groningen,1.0,NEE,9711,3.0,6264.0,...,103.0,3.987695,7.048178,5.039060,8.147042,91.0,4.147884,6.576391,5.140564,7.689187


In [12]:
for idx, row in tqdm(bu.iterrows(), total=bu.shape[0]):
    amenities = gpd.read_parquet(f"data/gm_amenities/amenities_{row['gemeentenaam']}.parquet")
    amenities = amenities[amenities.geometry.within(row.geometry)]
    for filter_i in [0, 1, 2]:
        
        # get and apply filter
        L0_filter, L1_filter = ec.getfilter(filter_i)
        
        amenities_f = amenities[~amenities.L0_category.isin(L0_filter)]
        if L1_filter:
            for key, value in L1_filter.items():
                amenities_f = amenities_f[
                    ~( (amenities_f.L0_category == key) & (amenities_f.L1_category.isin(value)) )
                ]
        
        bu.at[idx, f'L0_shannon_{filter_i}'] = _get_shannon_entropy(amenities_f.L0_category.values)
        bu.at[idx, f'L1_shannon_{filter_i}'] = _get_shannon_entropy(amenities_f.L1_category.values)
        
        for cat in L0_cats:
            bu.loc[idx, f'L1_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L0_category == cat])
        
        for cat in L1_cats:
            bu.loc[idx, f'L0_{filter_i}_count_{cat}'] = len(amenities_f[amenities_f.L1_category == cat])

    del amenities
    del amenities_f
    gc.collect()

bu.to_parquet(f'data/buurten_parts/calculated/buurten_c_{part}.parquet') 

100%|██████████| 1395/1395 [31:11<00:00,  1.34s/it] 
